In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load Model (FLAN-T5)
model_name = "google/flan-t5-large"  # change to base/small if you have GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
chat_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Chat Memory (sliding window)
class ChatMemory:
    def __init__(self, max_turns=5):
        self.max_turns = max_turns
        self.history = []

    def add(self, user, bot):
        self.history.append((user, bot))
        if len(self.history) > self.max_turns:
            self.history.pop(0)

    def get_context(self):
        context = ""
        for u, b in self.history:
            context += f"User: {u}\nBot: {b}\n"
        return context

# Chatbot Class
class Chatbot:
    def __init__(self):
        self.memory = ChatMemory(max_turns=3)

    def ask(self, query):
        # Prepare context
        context = self.memory.get_context()
        prompt = f"{context}User: {query}\nBot:"

        # Generate response
        response = chat_pipeline(
            prompt, max_length=200, truncation=True, do_sample=True
        )[0]['generated_text']

        # Clean response (remove extra "User:" if generated)
        if "User:" in response:
            response = response.split("User:")[0].strip()

        # Update memory
        self.memory.add(query, response)
        return response

# Run Chatbot (Terminal Loop)
if __name__ == "__main__":
    bot = Chatbot()
    print("🤖 Chatbot ready! Type your questions. Type '/exit' to stop.\n")

    while True:
        user_input = input("User: ")
        if user_input.strip().lower() in ["/exit", "exit", "quit"]:
            print("👋 Exiting chatbot. Goodbye!")
            break
        reply = bot.ask(user_input)
        print("Bot:", reply)


🤖 Chatbot ready! Type your questions. Type '/exit' to stop.



User:  what is AI?


Bot: Human-Computer Interaction


User:  what is example of AI?


Bot: Artificial Language


User:  /exit


👋 Exiting chatbot. Goodbye!
